<a href="https://colab.research.google.com/github/manekgnath/Machine-Learning/blob/main/ML_project_12_Predict_heart_Disease_Ensemble_ML_Algorithms_Bagging%2C_Boosting%2C_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [ ]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['Sex','RestingECG','ChestPainType','ExerciseAngina','ST_Slope'])
df = encoder.fit_transform(df)


In [ ]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,1,0.0,1,0
1,49,2,2,160,180,0,1,156,1,1.0,2,1
2,37,1,1,130,283,0,2,98,1,0.0,1,0
3,48,2,3,138,214,0,1,108,2,1.5,2,1
4,54,1,2,150,195,0,1,122,1,0.0,1,0


In [ ]:
y = df['HeartDisease']
x = df.drop('HeartDisease',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
bag_model = BaggingClassifier(estimator=tree,n_estimators=100,random_state=2)
bag_model.fit(x_train,y_train)
bag_model.score(x_test,y_test)

0.8315217391304348

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
y_pred = bag_model.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[72 17]
 [14 81]]
              precision    recall  f1-score   support

           0       0.84      0.81      0.82        89
           1       0.83      0.85      0.84        95

    accuracy                           0.83       184
   macro avg       0.83      0.83      0.83       184
weighted avg       0.83      0.83      0.83       184



In [ ]:
from sklearn.ensemble import RandomForestClassifier
tree = RandomForestClassifier()
bag_model_2 = BaggingClassifier(estimator=tree,n_estimators=100,random_state=2)
bag_model_2.fit(x_train,y_train)
bag_model_2.score(x_test,y_test)

0.8586956521739131

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model_1 = AdaBoostClassifier()
model_1.fit(x_train,y_train)
model_1.score(x_test,y_test)


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.8586956521739131

In [ ]:
y_pred_1 = model_1.predict(x_test)
print(confusion_matrix(y_test,y_pred_1))
print(classification_report(y_test,y_pred_1))

[[76 13]
 [13 82]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        89
           1       0.86      0.86      0.86        95

    accuracy                           0.86       184
   macro avg       0.86      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model_2 = GradientBoostingClassifier()
model_2.fit(x_train,y_train)
model_2.score(x_test,y_test)

0.8478260869565217

In [ ]:
y_pred_2 = model_2.predict(x_test)
print(confusion_matrix(y_test,y_pred_2))
print(classification_report(y_test,y_pred_2))

[[75 14]
 [14 81]]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84        89
           1       0.85      0.85      0.85        95

    accuracy                           0.85       184
   macro avg       0.85      0.85      0.85       184
weighted avg       0.85      0.85      0.85       184



Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

estimator = []
model1 = LogisticRegression()
estimator.append(('logistic',model1))
model2 = DecisionTreeClassifier()
estimator.append(('cart',model2))
model3 = SVC()
estimator.append(('svm',model3))

voting = VotingClassifier(estimator,voting='hard')
voting.fit(x_train,y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('cart', DecisionTreeClassifier()),
                             ('svm', SVC())])

In [ ]:
voting.score(x_test,y_test)

0.8478260869565217

In [ ]:
y_pred_4 = voting.predict(x_test)
print(confusion_matrix(y_test,y_pred_4))
print(classification_report(y_test,y_pred_4))

[[73 16]
 [12 83]]
              precision    recall  f1-score   support

           0       0.86      0.82      0.84        89
           1       0.84      0.87      0.86        95

    accuracy                           0.85       184
   macro avg       0.85      0.85      0.85       184
weighted avg       0.85      0.85      0.85       184

